In [1]:
import numpy as np
import pandas as pd

In [2]:
links = pd.read_csv('links_small.csv')
movies = pd.read_csv('movies_metadata.csv')

/var/folders/y9/369ghyk55mgbb3cn3tbv74440000gn/T/ipykernel_75539/2343206172.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


In [3]:
ratings = pd.read_csv('ratings_small.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [4]:
import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype 

user_index = ratings.userId.unique()
movies_index = ratings.movieId.unique()

rows = ratings['userId'].astype(CategoricalDtype(categories=user_index)).cat.codes 
cols = ratings['movieId'].astype(CategoricalDtype(categories=movies_index)).cat.codes 

matrix = sparse.csr_matrix((ratings.rating, (rows, cols)), shape=(len(user_index), len(movies_index)))

matrix = matrix.toarray()

In [5]:
for ind, score in enumerate(matrix[50]): 
    if score>=3:
        movie_id = int(links[links.movieId==movies_index[ind]].tmdbId.values[0])
        print(score, movies[movies['id']==str(movie_id)].original_title.values[0])


5.0 Stir of Echoes
5.0 Frequency
4.0 The Maltese Falcon
4.0 The Breakfast Club
4.0 Austin Powers: The Spy Who Shagged Me
4.0 Arachnophobia
3.0 The Mosquito Coast
5.0 Who Framed Roger Rabbit
5.0 The Iron Giant
5.0 The 13th Warrior
4.0 The Astronaut's Wife
3.0 Stigmata
5.0 Double Jeopardy
5.0 Stuart Little
4.0 Galaxy Quest
3.0 Desperately Seeking Susan
5.0 Entrapment
5.0 Lake Placid
5.0 Deep Blue Sea
4.0 The Thomas Crown Affair
5.0 Pacific Heights
4.0 Diner
4.0 Out of Africa
4.0 The General's Daughter
3.0 Hope Floats
4.0 Tom Jones
5.0 Run Silent, Run Deep


In [6]:
import implicit

model = implicit.als.AlternatingLeastSquares(factors=64, iterations=30)

model.fit(sparse.csr_matrix(matrix))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
model.user_factors

array([[ 0.1456562 , -0.22609325, -0.08109295, ...,  0.13658348,
         0.17909677,  0.11419078],
       [-0.48129943, -0.28109854,  0.45365852, ..., -0.72991306,
         1.9579935 ,  0.05197343],
       [ 0.01626042, -0.64669824,  1.3102347 , ..., -0.7088846 ,
         0.05020762,  1.0883926 ],
       ...,
       [ 0.05483034,  0.35578984,  0.9336086 , ..., -0.07237514,
         0.36479282,  0.8144462 ],
       [-0.20480546, -0.60882026,  1.4960787 , ..., -0.8383628 ,
         0.0149623 ,  1.249608  ],
       [-0.88748705, -0.88270146,  1.8505199 , ..., -1.2109996 ,
        -0.59309584,  0.69532543]], dtype=float32)

In [8]:
model.item_factors

array([[-0.0001673 ,  0.00242258, -0.01121366, ...,  0.03105991,
         0.05189091,  0.00477436],
       [ 0.03064282, -0.05440148, -0.00052225, ...,  0.00338218,
        -0.00122851, -0.03955786],
       [-0.03540804,  0.03527637,  0.00025152, ..., -0.00951132,
        -0.0101826 ,  0.01577788],
       ...,
       [ 0.00162506, -0.00482273, -0.00032332, ...,  0.00439688,
         0.00360549, -0.010636  ],
       [ 0.00082471, -0.0024502 , -0.00016462, ...,  0.00223475,
         0.00183168, -0.00540441],
       [ 0.00068062, -0.00012429,  0.00106485, ...,  0.00017621,
         0.00050912,  0.00020801]], dtype=float32)

In [9]:
recommendations = model.recommend(50, sparse.csr_matrix(matrix)[50])

In [10]:
recommendations

(array([119, 945, 118, 816, 947, 904, 521, 339, 550, 917], dtype=int32),
 array([0.5727198 , 0.56173766, 0.47739434, 0.47697037, 0.45804954,
        0.45561853, 0.4503094 , 0.43525553, 0.4259323 , 0.42253944],
       dtype=float32))

In [11]:
for ind in recommendations[0]: 
        movie_id = int(links[links.movieId==movies_index[ind]].tmdbId.values[0])
        print(movies[movies['id']==str(movie_id)].original_title.values[0])


The Sixth Sense
Arlington Road
Ghostbusters
Romy and Michele's High School Reunion
The Blair Witch Project
Elizabeth
Payback
A Bug's Life
Kingpin
The Faculty
